# Task 1

1. Estimate biases of the coins p1 and p2 by optimizing directly the marginal likelihood:
$$
p_1, p_2 = argmax_{p_1, p_2} \sum_Z P(X|Z, p1, p2)
$$

$$
P(X|Z, Q) = P_1(X|Z, Q) * P_2(X|Z, Q) *... * P_5(X|Z, Q)
$$

$$
P_i(X|Z, Q) ∝ p_{z_i}^{k_i} (1 - p_{z_i})^{10 - k_i}
$$

, where
* $i$ - number of experiment: 1 - 5
* $z_i$ - coin chosen in i-th experiment
* $p_{z_i}$ - probability of HEAD for coin $z_i$
* $k_i$ - number of times when we had HEAD in i-th experiment

In [1]:
import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from scipy import optimize

A = np.array([
    'HTTTHHTHTH',
    'HHHHTHHHHH',
    'HTHHHHHTHH',
    'HTHTTTHHTT',
    'THHHTHHHTH'
])
tosses = 5

In [2]:
import itertools

Z = list(itertools.product(range(2), repeat=tosses))

In [3]:
def prob(coins_id, p1, p2):
    flag_0 = coins_id == 0
    flag_1 = ~flag_0
    counts_1 = {c: np.sum([exp.count(c) for exp in A[flag_0]]) for c in ['H', 'T']}
    counts_2 = {c: np.sum([exp.count(c) for exp in A[flag_1]]) for c in ['H', 'T']}
    return p1**counts_1['H']*(1-p1)**counts_1['T'] * p2**counts_2['H']*(1-p2)**counts_2['T']

def likelihood(p):
  return jnp.sum( jnp.array([prob(jnp.array(z), p[0], p[1]) for z in Z]))

likelihood([0.8, 0.6])

Array(5.05329359e-13, dtype=float64)

In [4]:
from scipy import optimize
from scipy.optimize import Bounds
from scipy.integrate import dblquad

bounds = Bounds([0.0, 0.0], [1.0, 1.0])

res = dblquad(lambda x, y: likelihood([x, y]), 0, 1, 0, 1)
norm_coeff = 1.0 / res[0]

def loss(p):
  return -norm_coeff * likelihood(p)

p0 = np.random.uniform(low=0.0, high=1.0, size=(2,))
res = optimize.minimize(loss,
                        p0,
                        jac = jax.grad(loss),
                        hess=jax.hessian(loss),
                        method='trust-constr',
                        bounds=bounds)

res

           message: `gtol` termination condition is satisfied.
           success: True
            status: 1
               fun: -6.5625569443091685
                 x: [ 5.196e-01  7.968e-01]
               nit: 56
              nfev: 74
              njev: 22
              nhev: 22
          cg_niter: 46
      cg_stop_cond: 4
              grad: [-2.981e-08 -6.052e-08]
   lagrangian_grad: [-3.263e-09 -1.979e-09]
            constr: [array([ 5.196e-01,  7.968e-01])]
               jac: [<2x2 sparse matrix of type '<class 'numpy.float64'>'
                    	with 2 stored elements in Compressed Sparse Row format>]
       constr_nfev: [0]
       constr_njev: [0]
       constr_nhev: [0]
                 v: [array([ 2.654e-08,  5.854e-08])]
            method: tr_interior_point
        optimality: 3.262670632182317e-09
  constr_violation: 0.0
    execution_time: 27.31625747680664
         tr_radius: 1.0
    constr_penalty: 1.0
 barrier_parameter: 2.048000000000001e-09
 barrier_toleranc

In [6]:
print(f'Best estimation of coins biases p_1 and p_2 ={res.x}')

Best estimation of coins biases p_1 and p_2 =[0.51958312 0.79678907]


2. Estimate biases of the coins p1 and p2 by running iterative EM algorithm.

In [17]:
HT_stats = jnp.array([[A[i].count(c) for c in ['H', 'T']] for i in range(tosses)])
HT_stats

Array([[5, 5],
       [9, 1],
       [8, 2],
       [4, 6],
       [7, 3]], dtype=int64)

In [53]:
def get_probs(p0_init, p1_init):
  # 1. calculate probabilities that in each experiment certain coin was chosen
  probs = jnp.array([
      [p0_init**HT_stats[i][0] * (1 - p0_init)**HT_stats[i][1], p1_init**HT_stats[i][0] * (1 - p1_init)**HT_stats[i][1]]
      for i in range(tosses)
  ])
  probs = probs / probs.sum(axis=1, keepdims=True)

  # 2. calculate coefficients of log-likelihood of choosing coins across all tosses
  # logL = coeff[0]*ln(p_0) + coeff[1]*ln(1 - p_0) + coeff[2]*ln(p_1) + coeff[3]*ln(1 - p_1)
  coeff = jnp.array([
      [HT_stats[i][0]* probs[i][0], HT_stats[i][1]* probs[i][0], HT_stats[i][0]* probs[i][1], HT_stats[i][1]* probs[i][1]]
      for i in range(tosses)
  ])
  coeff = jnp.sum(coeff, axis=0)

  # 3. maximize log-likelihood for p_0 and p_1 by taking its partial deriviatives equal to zero
  p0_next = coeff[0] / (coeff[0] + coeff[1])
  p1_next = coeff[2] / (coeff[2] + coeff[3])

  log_likelihood = coeff[0]*jnp.log(p0_next) + coeff[1]*jnp.log(1-p0_next) + \
                   coeff[2]*jnp.log(p1_next) + coeff[3]*jnp.log(1-p1_next)

  return p0_next, p1_next, log_likelihood


In [63]:
def find_probs(p0_init, p1_init, tolerace=1e-6, max_iter=1000000):
  prev_ll = -1e10

  p0 = p0_init
  p1 = p1_init
  for step in range(max_iter):
    p0, p1, ll = get_probs(p0, p1)
    if abs(ll - prev_ll) < tolerace:
      print(f"Converged in {step + 1} iterations")
      return p0, p1
    prev_ll = ll

  print(f"No convergence, stopped by reaching max iteration count")
  return p0, p1

In [64]:
def run_em_algorithm(p0_init, p1_init):
  p0, p1 = find_probs(p0_init, p1_init)
  print(f"Optimal values are: [{p0}, {p1}]")

In [66]:
run_em_algorithm(0.5, 0.55)

Converged in 21 iterations
Optimal values are: [0.5195831374972835, 0.7967890630579104]


In [69]:
# lets try some othe initial values
run_em_algorithm(0.1, 0.9)
run_em_algorithm(0.01, 0.0001)

Converged in 17 iterations
Optimal values are: [0.5195830644314472, 0.7967890516788051]
Converged in 14 iterations
Optimal values are: [0.7967885064073513, 0.5195823690774435]


So, resulting values of probabilities p1 and p2 are almost the same as in the first step.